<a href="https://colab.research.google.com/github/Interpause/pseudo-text/blob/master/true_news_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install newspaper3k
import newspaper as news
from newspaper import Article, Source, Config
from newspaper.mthreading import NewsPool

In [0]:
from google.colab import drive #drive.flush_and_unmount()
import os
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/CSIT Internship/code')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Data cleaning 2.0

In [0]:
import pandas as pd
df = pd.read_pickle('./raw/myNews4.pkl')

In [0]:
import unicodedata
#normally also helps remove foreign languages
removables = ['AD']
def clean(x):
    x.title = ' '.join(w for w in unicodedata.normalize('NFKD',x.title.strip()).encode('ascii','ignore').decode('utf8','ignore').split() if w not in removables)
    x.text = ' '.join(w for w in unicodedata.normalize('NFKD',x.text.strip()).encode('ascii','ignore').decode('utf8','ignore').split() if w not in removables)
    if len(x.title) < 1 or len(x.text.split()) < config.MIN_WORD_COUNT:
        x.title = 'None'
        x.text = 'None'
    return x

In [0]:
df = df.dropna()

In [0]:
df = df.apply(clean,axis=1)[df.text != 'None']

In [0]:
df = df.drop_duplicates(subset=('text',))

In [0]:
if not 'uuid' in df:
    import uuid
    df = df.assign(uuid=tuple(uuid.uuid4() for x in range(len(df))))
    print('done')

In [0]:
df.to_pickle('./raw/myNews4.pkl')

In [0]:
import pandas as pd
df1 = pd.read_pickle('./raw/myNews.pkl')
df2 = pd.read_pickle('./raw/myNews2.pkl')
df3 = pd.read_pickle('./raw/myNews3.pkl')
df4 = pd.read_pickle('./raw/myNews4.pkl')

In [0]:
dff = pd.concat((df1,df2,df3,df4),ignore_index=True,sort=False)

In [0]:
dff.text = dff.text.str.lower()
dff.title = dff.title.str.lower()

In [0]:
subset_impt = ('text',)
dff = dff.drop_duplicates(subset=subset_impt).apply(clean,axis=1)[dff.text != 'None'].dropna(subset=subset_impt).drop_duplicates(subset=subset_impt).apply(clean,axis=1)[dff.text != 'None'].dropna(subset=subset_impt).reset_index(drop=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [0]:
tags_all = {}
def clean_tags(x):
    tags = set()
    if str(x) == 'nan': return tags
    for tag in x:
        if tag[-2:] == ' |': 
            tags.add(tag[:-2].lower())
        if ';' in tag:
            for y in tag.split(';'): tags.add(y.strip().lower())
    tags.discard('')
    for z in tags:
        tags_all[z] = tags_all.get(z,0) + 1
    return tags

In [0]:
dff.meta_tags = dff.meta_tags.apply(clean_tags)

In [0]:
#only about 700 articles have meta (human) keywords... use NER?
#keywords are very inconsistent, will take too long to standardize. either split further by spaces or embed them
dff.to_pickle('./raw/myNewsFinal.pkl')

In [0]:
from collections import OrderedDict
sorted(OrderedDict(tags_all).items(),key=lambda x: x[1],reverse=True)

[('donald trump', 33),
 ('general election', 30),
 ('global warming', 29),
 ('boris johnson', 28),
 ('christmas', 27),
 ('indybest', 27),
 ('nature', 21),
 ('climate', 21),
 ('spintronics', 20),
 ('neuroscience', 19),
 ('climate change', 18),
 ('physics', 17),
 ('black friday', 17),
 ('intelligence', 17),
 ('psychology', 16),
 ("alzheimer's", 16),
 ('pharmacology', 16),
 ('ecology', 16),
 ('materials science', 15),
 ('chemistry', 15),
 ('brexit', 14),
 ('cyber monday', 14),
 ('dementia', 14),
 ('champions league', 13),
 ('anthony joshua', 13),
 ('mental health', 13),
 ('black friday 2019', 13),
 ('computers and internet', 13),
 ('astronomy', 13),
 ('brain injury', 13),
 ('brain-computer interfaces', 13),
 ('sun', 13),
 ('inorganic chemistry', 13),
 ('nanotechnology', 13),
 ('graphene', 13),
 ('diseases and conditions', 13),
 ('jeremy corbyn', 12),
 ('biodiversity', 12),
 ('bird flu', 12),
 ('air pollution', 12),
 ('cancer', 12),
 ('biology', 12),
 ('environmental issues', 12),
 ('air q

In [0]:
dff.groupby('outlet').count()
#deal with foreign languages that utilizes ascii later... (url filtering)
#newspaper3k didnt parse keywords for reuters properly

,url,date,category,title,text,meta,uuid,meta_tags
outlet,,,,,,,,
abcnews.go.com,226,226,226,226,226,226,226,226
ap.org,18,18,18,18,18,18,18,18
asiaone.com,353,353,353,353,353,353,353,353
bbc.com,209,209,209,209,209,209,209,209
businessinsider.sg,706,706,706,706,706,706,706,706
businesstimes.com.sg,492,492,492,492,492,492,492,492
cbsnews.com,442,442,442,442,442,442,442,442
channelnewsasia.com,109,109,109,109,109,109,109,109
cnbc.com,267,267,267,267,267,267,267,267


In [0]:
dff

,url,date,outlet,category,title,text,meta,uuid,meta_tags
0,http://www.channelnewsasia.com/news/cna938,-1.000000e+00,channelnewsasia.com,https://www.channelnewsasia.com/news/cna938,CNA938,6AM to 7AM The Weekly with Keith De Souza and ...,"{""viewport"": ""width=device-width, initial-scal...",5fa0da01-b2b4-4f74-904a-349eebfe6923,{}
1,https://www.channelnewsasia.com/news/rss,-1.000000e+00,channelnewsasia.com,https://www.channelnewsasia.com/news/rss,"Breaking news, latest Singapore, Asia and worl...",RSS (Really Simple Syndication) gives you the ...,"{""viewport"": ""width=device-width, initial-scal...",832489e6-ac8e-4166-85cd-5f368dd39375,{}
2,http://www.channelnewsasia.com/news/singapore/...,-1.000000e+00,channelnewsasia.com,https://www.channelnewsasia.com/news/singapore...,Companies hunt for talent as Singapore tech se...,"SINGAPORE: As a child, Huang Wei Long liked to...","{""viewport"": ""width=device-width, initial-scal...",a8031ffb-9db7-42dc-988b-24ec53dc63a5,{}
3,http://www.channelnewsasia.com/news/singapore/...,-1.000000e+00,channelnewsasia.com,https://www.channelnewsasia.com/news/singapore...,PIE viaduct collapse: Jail for engineer who kn...,SINGAPORE: An engineer overseeing works who di...,"{""viewport"": ""width=device-width, initial-scal...",94796e25-6ad2-49ed-b927-dca8bf574134,{}
4,http://www.channelnewsasia.com/news/singapore/...,-1.000000e+00,channelnewsasia.com,https://www.channelnewsasia.com/news/singapore...,"The Big Read: After the sound and fury, the du...",Have the actions of some black sheep caused au...,"{""viewport"": ""width=device-width, initial-scal...",f71dfba8-abab-4a0e-820f-8ffe5a87c951,{}
...,...,...,...,...,...,...,...,...,...
14000,http://de.reuters.com/article/deutschland-spor...,1.576056e+09,reuters.com,https://de.reuters.com/article/deutschland-spo...,Blatt - Warenhausriese Karstadt Kaufhof uberni...,A warehouse of the German department store cha...,"{""robots"": ""index, follow"", ""viewport"": ""width...",7a230e27-c153-4dd4-a8f9-b5722dfc4925,{}
14001,http://de.reuters.com/article/usa-milit-r-erde...,1.576057e+09,reuters.com,https://de.reuters.com/article/usa-milit-r-erd...,US-Militar will Bau von Seltene-Erden-Fabriken...,People walk past the U.S. Capitol Building in ...,"{""robots"": ""index, follow"", ""viewport"": ""width...",cfa1fc59-18e5-4da6-aa99-dff58e7306ec,{}
14002,http://de.reuters.com/article/sterreich-regier...,1.576057e+09,reuters.com,https://de.reuters.com/article/sterreich-regie...,Osterreichs Ubergangskanzlerin hofft auf baldi...,Austria's Chancellor Brigitte Bierlein arrives...,"{""robots"": ""index, follow"", ""viewport"": ""width...",9bd83a59-a68d-4537-807e-d8d47733535d,{}
14003,http://de.reuters.com/article/china-journalism...,1.576053e+09,reuters.com,https://de.reuters.com/article/china-journalis...,Organisation - China hat 2019 die meisten Jour...,A graduate stands behind the Chinese flag duri...,"{""robots"": ""index, follow"", ""viewport"": ""width...",8c464c42-e45f-4f2a-b533-2e8a7322771d,{}


In [0]:
for _,row in dff.iterrows():
    if 'indybest' in row.meta_tags:
        print(row.title)

Best Christmas wreaths 2019: Real and artificial arrangements for your door
11 best log burners that are better for the environment
9 best men's ski and snowboard pants to stay protected on the slopes
8 best Christmas pudding alternatives to impress your guests with
Best Christmas beauty gifts under 50: Make-up, skincare, candles and more
15 best Christmas gifts for teenagers that they'll actually use
Best Cyber Monday Amazon deals: Best offers live to buy today
Cyber Monday Apple deals UK: Apple Watch, Airpods, iPhone offers to buy today
Best Cyber Monday laptop deals: Apple, Microsoft and more of the best offers to buy today
Best Cyber Monday kids' toys deals: Offers on Lego, Harry Potter, Frozen and more
Best Cyber Monday 2019 furniture deals from Zara Home, Made, Swoon and more
Best Cyber Monday TV deals: Top offers from Currys PC World, Amazon and Argos today
Best Cyber Monday console deals 2019: Nintendo Switch, PS4, Xbox One and more gaming deals to buy today
Best Argos, John Le

# Scrape

In [0]:
config = Config()
config.memoize_articles = False
config.language = 'en'
config.fetch_images = False
config.request_timeout = 60
config.thread_request_timeout = 60
config.follow_meta_refresh = True
config.MIN_WORD_COUNT = 150
config.MIN_SENT_COUNT = 4
root = 'http://www.'
config.browser_user_agent = 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'

In [0]:
with open('./news_outlets.txt','r') as f:
    outlets = f.read().splitlines()

In [0]:
outlets = [
            'channelnewsasia.com',
            'straitstimes.com',
            'ap.org',
            'reuters.com',
            'todayonline.com',
            'tnp.sg',
            'bbc.com',
            'economist.com',
            'businesstimes.com.sg',
            'businessinsider.sg',
            'nytimes.com',
            'independent.co.uk',
            'express.co.uk',
            'mirror.co.uk',
            'cnbc.com',
            'telegraph.co.uk',
            'washingtontimes.com',
            'washingtonpost.com',
            'nbcnews.com',
            'cnn.com',
            'time.com',
            'latimes.com',
            'nydailynews.com',
            'newsroompost.com',
            'npr.org',
            'nationalgeographic.com',
            'medium.com',
            'discovery.com',
            'sciencedaily.com',
            'phys.org',
            'dailymail.co.uk',
            'cbsnews.com',
            'huffpost.com',
            'abcnews.go.com',
            'dailymail.co.uk',
            'forbes.com',
            'scientificamerican.com',
            'today.com',
            'pbs.org',
            'news.com.au',
            'theatlantic.com',
            'bloomberg.com',
            'foreignaffairs.com',
            'wsj.com',
            'newyorker.com',
            'fortune.com',
            'usatoday.com',
            'asiaone.com',
            'nst.com.my',
            'heraldsun.com.au',
            'nzherald.co.nz'
            ]

In [0]:
papers = {}
for outlet in outlets:
    print(outlet)
    catemap = {}
    paper = news.build(root+outlet,config)
    paper.build()
    papers[outlet] = paper

channelnewsasia.com
straitstimes.com
ap.org
reuters.com
todayonline.com
tnp.sg
bbc.com
economist.com
businesstimes.com.sg
businessinsider.sg
nytimes.com
independent.co.uk
express.co.uk
mirror.co.uk
cnbc.com
telegraph.co.uk
washingtontimes.com
washingtonpost.com
nbcnews.com
cnn.com
time.com
latimes.com
nydailynews.com
newsroompost.com
npr.org
nationalgeographic.com
medium.com
discovery.com
sciencedaily.com
phys.org
dailymail.co.uk
cbsnews.com
huffpost.com
abcnews.go.com
dailymail.co.uk
forbes.com
scientificamerican.com
today.com
pbs.org
news.com.au
theatlantic.com
bloomberg.com
foreignaffairs.com
wsj.com
newyorker.com
fortune.com
usatoday.com
asiaone.com
nst.com.my
heraldsun.com.au
nzherald.co.nz


In [0]:
news_pool = NewsPool(config)

In [0]:
news_pool.set(papers.values(), threads_per_source=1)

In [0]:
#%config Application.log_level="DEBUG"
import datetime,dateutil,json
import pandas as pd
procPapers = pd.DataFrame({},columns=('url','date','outlet','category','title','text','meta_tags','meta'))
not_done = True
while not_done:
    not_done = False
    for outlet,source in papers.items():
        for article in source.articles:
            #resolves articles while threads are still downloading them
            if article.is_parsed: continue
            if article.download_state == 0:
                not_done = True
                continue
            if article.download_state == 1: continue #1 is the enum value of failed
            if article.is_media_news(): continue #if its video/picture based

            article.parse()
            article_dat = {}
            article_dat['outlet'] = outlet
            article_dat['meta_tags'] = set(article.meta_keywords + list(article.tags))
            article_dat['url'] = article.url
            article_dat['meta'] = json.dumps(article.meta_data)
            article_dat['category'] = article.canonical_link
            article_dat['title'] = article.title
            article_dat['text'] = article.text

            if type(article.publish_date) == datetime.datetime:
                article_dat['date'] = article.publish_date.timestamp()
            else:
                try:
                    article_dat['date'] = int(dateutil.parser.parse(article.meta_data.get('pubdate','None')))
                except:
                    article_dat['date'] = -1
            procPapers = procPapers.append(article_dat,ignore_index=True)

            #save ram
            article.html = ''
            article.article_html = ''
            article.doc = None
            article.top_node = None
            article.clean_top_node = None
            article.clean_doc = None
            


procPapers.to_pickle('./raw/myNews4.pkl')

Building prefix dict from /usr/local/lib/python3.6/dist-packages/jieba/dict.txt ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.6617436408996582 seconds.
Prefix dict has been built succesfully.


Newspaper's keyword extraction is frequency not tfidf... <br>
Newspaper's summarization is sentence extraction based on frequency of key word....<br>
In conclusion: Use own extractor & paraphraser? or summarizer?<br>
final size: 12k perhaps was ip blocked or smth (evident from the article count from certain sites), create workaround.